# SHAP feature selection #
## Code to select feature with combination of LightGBM and SHAP ##

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [29]:
number_of_folds = 5 # this sets global setting of which how many bootstraps to use
repeats = 10
#first round of optimization
lgb_params = {
 'boosting_type': 'gbdt',
 'objective': 'regression',
 'lambda_l1': 3.5389955270605298,
 'lambda_l2': 1.6581000555478702,
 'metric': ["l1", "l2", "huber"],
 'max_leaves': 20,
 'max_depth': 8,
 'feature_fraction': 0.537151250668454,
 'bagging_fraction': 0.6407512594833191,
 'learning_rate': 0.06627424358150694,
 'min_data_in_leaf': 3,
 'drop_rate': 0.16918140176888039}
debug_local = True #to use local version

In [30]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '..', '..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.models import Metrics
from yspecies.selection import ShapSelector

In [33]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [34]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [35]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12340)",12340,39,445,"(12340, 2)","(40, 19)"


## Setting up SHAP selection pipeline ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [36]:
partition_params = PartitionParameters(number_of_folds, 0, 2, [],  42)


In [37]:
partition_shap_pipe = Pipeline([
    ("partitioner", DataPartitioner()),
    ('prepare_for_partitioning', TupleWith(lgb_params)),
    ("shap_computation", ShapSelector())
]
)
repeated_cv =  Repeat(partition_shap_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))

In [38]:
selection_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("partition_shap", repeated_cv)]
    )

## Setting up features to select ##

In [39]:
selection = select_lifespan = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])

In [40]:
select_lifespan = selection
select_mass = replace(selection, to_predict = "mass_g")
select_gestation = replace(selection, to_predict = "gestation")
select_mtgc = replace(selection, to_predict = "mtgc")

# First stage selection #

In [41]:
stage_one_lifespan = selection_pipeline.fit_transform((data, select_lifespan))
stage_one_lifespan

Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 3.88913	valid_0's l2: 34.4348	valid_0's huber: 3.15574
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 3.88913	valid_0's l2: 34.4348	valid_0's huber: 3.15574
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 5.07552	valid_0's l2: 59.3379	valid_0's huber: 4.23463
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 5.07552	valid_0's l2: 59.3379	valid_0's huber: 4.23463
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 3.49855	valid_0's l2: 33.6041	valid_0's huber: 2.82957
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 3.49855	valid_0's l2: 33.6041	valid_0's huber: 2.82957
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 3.96945	valid_0's l2: 26.2622	valid_0's huber: 3.20037
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 3.96945	valid_0's l

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[25]	valid_0's l1: 8.27336	valid_0's l2: 152.951	valid_0's huber: 7.04988
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[140]	valid_0's l1: 3.66283	valid_0's l2: 98.9854	valid_0's huber: 3.02306
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 2.58503	valid_0's l2: 18.7909	valid_0's huber: 2.01481
Did not meet early stopping. Best iteration is:
[199]	valid_0's l1: 2.58489	valid_0's l2: 18.7909	valid_0's huber: 2.01475
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 3.33061	valid_0's l2: 26.1408	valid_0's huber: 2.67011
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 3.33061	valid_0's l2: 26.1408	valid_0's huber: 2.67011
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 4.31224	valid_0's l2: 72.2134	valid_0's huber: 3.54395
Did n

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[30]	valid_0's l1: 8.04201	valid_0's l2: 130.572	valid_0's huber: 6.85058
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 2.12516	valid_0's l2: 18.957	valid_0's huber: 1.63424
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 2.12516	valid_0's l2: 18.957	valid_0's huber: 1.63424
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 3.17125	valid_0's l2: 27.2013	valid_0's huber: 2.55549
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 3.17125	valid_0's l2: 27.2013	valid_0's huber: 2.55549
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 2.06907	valid_0's l2: 12.436	valid_0's huber: 1.54729
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 2.06907	valid_0's l2: 12.436	valid_0's huber: 1.54729
Training until validation scores don't improve for 10 rou

...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[127]	valid_0's l1: 2.92949	valid_0's l2: 45.7961	valid_0's huber: 2.33054
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[187]	valid_0's l1: 2.39544	valid_0's l2: 26.3228	valid_0's huber: 1.86394
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 4.64778	valid_0's l2: 118.561	valid_0's huber: 3.90177
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 4.64778	valid_0's l2: 118.561	valid_0's huber: 3.90177
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[72]	valid_0's l1: 7.75601	valid_0's l2: 139.923	valid_0's huber: 6.62673
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[43]	valid_0's l1: 4.03212	valid_0's l2: 37.2597	valid_0's huber: 3.23924


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[152]	valid_0's l1: 3.51657	valid_0's l2: 83.9322	valid_0's huber: 2.87289
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's l1: 10.4004	valid_0's l2: 189.667	valid_0's huber: 8.96228
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[153]	valid_0's l1: 3.82282	valid_0's l2: 37.8346	valid_0's huber: 3.12812
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[176]	valid_0's l1: 3.09031	valid_0's l2: 45.8416	valid_0's huber: 2.4719
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 21.3302	valid_0's l2: 1927.37	valid_0's huber: 18.8882
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 21.3302	valid_0's l2: 1927.37	valid_0's huber: 18.8882


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


[<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
 <style scoped>
     .dataframe tbody tr th:only-of-type {
         vertical-align: middle;
     }
 
     .dataframe tbody tr th {
         vertical-align: top;
     }
 
     .dataframe thead th {
         text-align: right;
     }
 </style>
 <table border="1" class="dataframe">
   <thead>
     <tr style="text-align: right;">
       <th></th>
       <th>symbol</th>
       <th>gain_score_to_lifespan</th>
       <th>kendall_tau_to_lifespan</th>
     </tr>
     <tr>
       <th>ensembl_id</th>
       <th></th>
       <th></th>
       <th></th>
     </tr>
   </thead>
   <tbody>
     <tr>
       <th>ENSG00000276293</th>
       <td>PIP4K2B</td>
       <td>595468.075</td>
       <td>0.023</td>
     </tr>
     <tr>
       <th>ENSG00000083896</th>
       <td>YTHDC1</td>
       <td>417638.774</td>
       <td>0.173</td>
     </tr>
     <tr>
       <th>ENSG00000060762</th>
       <td>MPC1</td>
       <td>135515.969</td>
       <td>-0.328</td>
     </tr>
     <tr>
       <th>ENSG00000167515</th>
       <td>TRAPPC2L</td>
       <td>127575.460</td>
       <td>-0.318</td>
     </tr>
     <tr>
       <th>ENSG00000204498</th>
       <td>NFKBIL1</td>
       <td>50323.852</td>
       <td>-0.276</td>
     </tr>
     <tr>
       <th>ENSG00000010219</th>
       <td>DYRK4</td>
       <td>37149.404</td>
       <td>0.200</td>
     </tr>
     <tr>
       <th>ENSG00000105672</th>
       <td>ETV2</td>
       <td>35001.381</td>
       <td>0.676</td>
     </tr>
     <tr>
       <th>ENSG00000119616</th>
       <td>FCF1</td>
       <td>23337.054</td>
       <td>-0.217</td>
     </tr>
     <tr>
       <th>ENSG00000129187</th>
       <td>DCTD</td>
       <td>17696.302</td>
       <td>0.650</td>
     </tr>
     <tr>
       <th>ENSG00000188763</th>
       <td>FZD9</td>
       <td>17617.694</td>
       <td>0.054</td>
     </tr>
     <tr>
       <th>ENSG00000109099</th>
       <td>PMP22</td>
       <td>9835.958</td>
       <td>-0.143</td>
     </tr>
     <tr>
       <th>ENSG00000170835</th>
       <td>CEL</td>
       <td>9568.904</td>
       <td>0.503</td>
     </tr>
     <tr>
       <th>ENSG00000173567</th>
       <td>ADGRF3</td>
       <td>8174.140</td>
       <td>0.487</td>
     </tr>
     <tr>
       <th>ENSG00000188747</th>
       <td>NOXA1</td>
       <td>5744.466</td>
       <td>0.722</td>
     </tr>
     <tr>
       <th>ENSG00000100650</th>
       <td>SRSF5</td>
       <td>5613.632</td>
       <td>0.462</td>
     </tr>
     <tr>
       <th>ENSG00000135845</th>
       <td>PIGC</td>
       <td>4834.881</td>
       <td>0.351</td>
     </tr>
     <tr>
       <th>ENSG00000157343</th>
       <td>ARMC12</td>
       <td>4753.970</td>
       <td>0.660</td>
     </tr>
     <tr>
       <th>ENSG00000065268</th>
       <td>WDR18</td>
       <td>4676.819</td>
       <td>-0.655</td>
     </tr>
     <tr>
       <th>ENSG00000129988</th>
       <td>LBP</td>
       <td>4282.956</td>
       <td>-0.547</td>
     </tr>
     <tr>
       <th>ENSG00000066923</th>
       <td>STAG3</td>
       <td>4096.389</td>
       <td>0.484</td>
     </tr>
     <tr>
       <th>ENSG00000134489</th>
       <td>HRH4</td>
       <td>3689.356</td>
       <td>0.597</td>
     </tr>
     <tr>
       <th>ENSG00000116489</th>
       <td>CAPZA1</td>
       <td>3583.179</td>
       <td>0.020</td>
     </tr>
     <tr>
       <th>ENSG00000171121</th>
       <td>KCNMB3</td>
       <td>3494.629</td>
       <td>0.568</td>
     </tr>
     <tr>
       <th>ENSG00000142541</th>
       <td>RPL13A</td>
       <td>3257.056</td>
       <td>0.020</td>
     </tr>
     <tr>
       <th>ENSG00000128739</th>
       <td>SNRPN</td>
       <td>3061.803</td>
       <td>-0.438</td>
     </tr>
     <tr>
       <th>ENSG00000101190</th>
       <td>TCFL5</td>
       <td>3014.514</td>
       <td>0.640</td>
     </tr>
     <tr>
       <th>ENSG00000142002</th>
       <td>DPP9</td>
       <

In [42]:
from yspecies.results import FeatureSummary
summary = FeatureSummary(stage_one_lifespan)
stage_one_lifespan[0]

<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>symbol</th>
      <th>gain_score_to_lifespan</th>
      <th>kendall_tau_to_lifespan</th>
    </tr>
    <tr>
      <th>ensembl_id</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ENSG00000276293</th>
      <td>PIP4K2B</td>
      <td>595468.075</td>
      <td>0.023</td>
    </tr>
    <tr>
      <th>ENSG00000083896</th>
      <td>YTHDC1</td>
      <td>417638.774</td>
      <td>0.173</td>
    </tr>
    <tr>
      <th>ENSG00000060762</th>
      <td>MPC1</td>
      <td>135515.969</td>
      <td>-0.328</td>
    </tr>
    <tr>
      <th>ENSG00000167515</th>
      <td>TRAPPC2L</td>
      <td>127575.460</td>
      <td>-0.318</td>
    </tr>
    <tr>
      <th>ENSG00000204498</th>
      <td>NFKBIL1</td>
      <td>50323.852</td>
      <td>-0.276</td>
    </tr>
    <tr>
      <th>ENSG00000010219</th>
      <td>DYRK4</td>
      <td>37149.404</td>
      <td>0.200</td>
    </tr>
    <tr>
      <th>ENSG00000105672</th>
      <td>ETV2</td>
      <td>35001.381</td>
      <td>0.676</td>
    </tr>
    <tr>
      <th>ENSG00000119616</th>
      <td>FCF1</td>
      <td>23337.054</td>
      <td>-0.217</td>
    </tr>
    <tr>
      <th>ENSG00000129187</th>
      <td>DCTD</td>
      <td>17696.302</td>
      <td>0.650</td>
    </tr>
    <tr>
      <th>ENSG00000188763</th>
      <td>FZD9</td>
      <td>17617.694</td>
      <td>0.054</td>
    </tr>
    <tr>
      <th>ENSG00000109099</th>
      <td>PMP22</td>
      <td>9835.958</td>
      <td>-0.143</td>
    </tr>
    <tr>
      <th>ENSG00000170835</th>
      <td>CEL</td>
      <td>9568.904</td>
      <td>0.503</td>
    </tr>
    <tr>
      <th>ENSG00000173567</th>
      <td>ADGRF3</td>
      <td>8174.140</td>
      <td>0.487</td>
    </tr>
    <tr>
      <th>ENSG00000188747</th>
      <td>NOXA1</td>
      <td>5744.466</td>
      <td>0.722</td>
    </tr>
    <tr>
      <th>ENSG00000100650</th>
      <td>SRSF5</td>
      <td>5613.632</td>
      <td>0.462</td>
    </tr>
    <tr>
      <th>ENSG00000135845</th>
      <td>PIGC</td>
      <td>4834.881</td>
      <td>0.351</td>
    </tr>
    <tr>
      <th>ENSG00000157343</th>
      <td>ARMC12</td>
      <td>4753.970</td>
      <td>0.660</td>
    </tr>
    <tr>
      <th>ENSG00000065268</th>
      <td>WDR18</td>
      <td>4676.819</td>
      <td>-0.655</td>
    </tr>
    <tr>
      <th>ENSG00000129988</th>
      <td>LBP</td>
      <td>4282.956</td>
      <td>-0.547</td>
    </tr>
    <tr>
      <th>ENSG00000066923</th>
      <td>STAG3</td>
      <td>4096.389</td>
      <td>0.484</td>
    </tr>
    <tr>
      <th>ENSG00000134489</th>
      <td>HRH4</td>
      <td>3689.356</td>
      <td>0.597</td>
    </tr>
    <tr>
      <th>ENSG00000116489</th>
      <td>CAPZA1</td>
      <td>3583.179</td>
      <td>0.020</td>
    </tr>
    <tr>
      <th>ENSG00000171121</th>
      <td>KCNMB3</td>
      <td>3494.629</td>
      <td>0.568</td>
    </tr>
    <tr>
      <th>ENSG00000142541</th>
      <td>RPL13A</td>
      <td>3257.056</td>
      <td>0.020</td>
    </tr>
    <tr>
      <th>ENSG00000128739</th>
      <td>SNRPN</td>
      <td>3061.803</td>
      <td>-0.438</td>
    </tr>
    <tr>
      <th>ENSG00000101190</th>
      <td>TCFL5</td>
      <td>3014.514</td>
      <td>0.640</td>
    </tr>
    <tr>
      <th>ENSG00000142002</th>
      <td>DPP9</td>
      <td>2930.752</td>
      <td>-0.592</td>
    </tr>
    <tr>
      <th>ENSG00000184459</th>
      <td>BPIFC</td>
      <td>2668.091</td>
      <td>0.727</td>
    </tr>
    <tr>
      <th>ENSG00

In [15]:
summary.selected.sort_values("kendall_tau_0", ascending=False)["symbol"].values

array(['CALCOCO2', 'MPG', 'NOXA1', 'ETV2', 'CEL', 'ARMC12', 'HRH4',
       'DCTD', 'KCNMB3', 'ADGRF3', 'TIMP1', 'TSPAN10', 'STAG3', 'GUCY2C',
       'DNAJB6', 'LIN7B', 'TRAPPC2L', 'MPC1', 'CTBS', 'CA3', 'NDUFA6',
       'LBP', 'PUSL1', 'MPC2', 'SNRPN', 'ADPRM', 'DPP9', 'WDR18'],
      dtype=object)

In [73]:
"gain" if len([c for c in summary.results[0].selected.columns if "gain" in c])>0 else "shap"

'gain'